# Preprocessing: segmentation of cars within the dataset
Here, we will go through our dataset of car images and use Ultralytics' YOLO to detect the cars using image segmentation and cut them out of the pictures. This will act as a way of 'cleaning' our data and reduce the noise of the input of our network later on.

In [20]:
# import dependencies
from ultralytics import YOLO
import cv2
import numpy as np
import os

In [21]:
# define folders
dir_segmentedcars = os.path.join('Data','segmented_cars')
dir_carsoriginal = os.path.join('Data', 'Cars')

In [22]:
def get_segmented_image(imgpath):
    # Load the segmentation model
    segmentation_model = YOLO('YOLO_models\yolov8n-seg.pt')
    
    # perform the segmentation on the image
    results = segmentation_model.predict(imgpath, retina_masks=True)

    # get only the segmentation-highlighted objects from the image using the obtained masks
    # from https://github.com/ultralytics/ultralytics/issues/1411
    for result in results:
        mask = result.masks.cpu().numpy()
        masks = mask.masks.astype(bool)
        ori_img = result.orig_img
        for m in masks:
            # initialise new segmented image as numpy array
            segmented_img = np.zeros_like(ori_img, dtype=np.uint8)
            # overlay mask on original image to get segmented image,
            # save this as segmented_img
            segmented_img[m] = ori_img[m]
    
    return segmented_img

In [48]:
# function that performs the segmentation of the car
def save_segmented_image(imgpath_orig, segm_folder):
    # get img filename and folder from imgpath
    orig_folder, imgfilename = os.path.split(imgpath_orig)
    # get image name and extension from the filename
    imgname, fileextension = ''.join(imgfilename.split('.')[:-1]), imgfilename.split('.')[-1]
    # generate relevant filepaths
    imgpath_orig = os.path.join(orig_folder, '.'.join((imgname, fileextension)))
    imgpath_segm = os.path.join(segm_folder, '.'.join((imgname + '_segm', fileextension)))
    
    # perform segmentation
    segmented_img = get_segmented_image(imgpath_orig)

    # save segmented image
    cv2.imwrite(imgpath_segm, segmented_img)

In [49]:
# test these functions
orig_imgpath = os.path.join(dir_carsoriginal, 'Audi', '_mg_2867.jpg')
save_segmented_image(orig_imgpath, dir_segmentedcars)


image 1/1 c:\Users\emiel\OneDrive\Personal projects\Machine Learning\car-recognition-ML\Data\Cars\Audi\_mg_2867.jpg: 384x640 1 car, 128.6ms
Speed: 2.0ms preprocess, 128.6ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
